Transformer From Scratch 
========================

Reading along Dan Jurafsky and James H. Martin's [Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book, I decided to follow through Chapter 8 of their book to implement a Transformer using Pytorch. It is my goal to have a working transformer which I can use to train on the guitar dataset. I know some linear algebra, the book essentially gives the entire algorithm in terms of linear algebra, and pytorch provides a nice but still very informative abstractions for doing linear algebra. I had no reason not to pursue this project on top of whatever I proposed to do initially. 

## 

Attention Layer
---------------

At the heart of Transformer is the **attention layer**. It is a mechanism that allows words(tokens) to gain contextual meaning from their surrounding words(tokens). It can have multiple **"heads"**, where each "head" can be thought of as a specialist who asks particular set of questions given some data. For instance, one head could focus solely on grammar while another could instead focus on sentiments (even though that might not be exactly what occurs under the hood).

Each head's job, then, is to ask the right kinds of *questions* to choose which of previous words it has seen matters the most to the current word. To do this, each head consists of three main components: **Query**, **Key**, and **Value** weight matrices. 

<!-- 
    essentially, what it is at the end of the day is weighted sum, but it's obviously lot more complicated than that
    don't forget to write out the equations that I have referenced
    maybe throw in some pictures
    say something about how masking and softmax is used to determine what key's to focus on
    also explain how results from different heads are consolidated at the end
--!>

In [7]:
from myTransformer import *
batch_size = 10
N = 10
model_dim = 24
num_heads = 4
key_dim = 3

M = 8
X = torch.rand((batch_size, N, model_dim)) # batch_size is 10, 3 words represented as dim (1, 4) tensors
Y = torch.rand((batch_size, M, model_dim)) # 3 words represented as dim (1, 4) tensors
mask = torch.tensor([[0 if i>= j else -torch.inf for j in range(N)] for i in range(N)])

multihead_attention = AttentionLayer(model_dim=model_dim, key_dim=key_dim, num_heads=num_heads)
multihead_attention(X, H_enc=Y, mask=mask).shape
#multihead_attention.to("cuda")
#multihead_attention(X.to("cuda"), Y.to("cuda"), mask=mask.to("cuda"))

torch.Size([10, 10, 24])

In [3]:
encoder_block = TransformerBlock(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=8, num_heads=num_heads)
decoder_block = TransformerBlock(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=8, num_heads=num_heads, cross_attention=True)
encoder_block(X,mask=mask)
decoder_block(X, Y)

tensor([[[ 0.6877,  1.2730,  1.0515,  ...,  0.2955,  0.2538,  0.7861],
         [ 1.1157,  1.3042, -0.3177,  ..., -0.1759,  0.4098, -0.1904],
         [ 0.7320,  0.4290,  0.6832,  ..., -0.0232,  0.3461,  0.1297],
         ...,
         [ 0.8951,  1.2577,  0.5012,  ...,  0.3529,  0.8690,  0.9116],
         [ 0.6383,  0.8001,  0.7895,  ...,  0.2262, -0.1620,  0.1951],
         [ 0.8677,  0.8694,  0.0960,  ...,  0.1690,  0.1500,  0.7893]],

        [[ 1.0179,  0.4911, -0.2982,  ..., -0.4886, -0.5350, -0.0442],
         [ 0.4181,  0.6220, -0.5301,  ..., -0.0118, -0.2998,  0.5791],
         [ 0.5394,  0.5750, -0.0586,  ..., -0.0286, -0.0823,  0.2187],
         ...,
         [ 0.4613,  1.0546,  0.4756,  ...,  0.3716,  0.2181,  0.5120],
         [ 0.7842,  0.8083,  0.3636,  ...,  0.8225,  0.6932,  0.8008],
         [ 0.2404,  1.1777, -0.2595,  ...,  0.3520,  0.5705,  0.8119]],

        [[ 0.7139,  0.6964,  0.6693,  ...,  0.5268, -0.1088,  0.2713],
         [ 0.2072,  1.0206, -0.0790,  ...,  1

In [13]:
stack = TransformerStack(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=8, num_heads=num_heads, num_stack=9)
stack.state_dict()
stack.train()
stack(X)

tensor([[[-0.4209,  0.6290,  1.7242,  ...,  0.9101,  0.2323, -0.9010],
         [-0.2763,  1.1280,  2.3604,  ...,  0.1779,  0.9750,  0.3167],
         [-0.2118,  1.9162,  1.6614,  ...,  1.1163,  0.2874, -0.3483],
         ...,
         [-0.3239,  0.9006,  1.4460,  ...,  0.3262,  0.8036, -0.0161],
         [-0.4742,  1.6472,  1.4774,  ...,  0.3226,  1.2245, -0.5092],
         [-0.2354,  1.5815,  1.0622,  ...,  0.3616,  0.9162, -1.3798]],

        [[-0.2367,  0.1023,  0.4686,  ...,  0.8498,  0.3110, -2.0294],
         [-0.8515,  0.3374,  1.0734,  ...,  0.2413,  0.9928, -1.2730],
         [-0.2414,  0.9426,  1.4753,  ...,  0.8885,  0.4580, -1.2081],
         ...,
         [-0.0348,  0.0921,  0.9640,  ...,  0.8393,  0.4511, -1.2873],
         [-0.2303,  0.3779,  2.0470,  ..., -0.3713,  0.3623, -0.2285],
         [ 0.0509, -0.0437,  1.8964,  ...,  0.4547,  0.0988, -0.6494]],

        [[ 0.2625,  1.1482,  0.6794,  ...,  1.1048,  1.1857, -0.8462],
         [ 0.3365,  0.9428,  1.6494,  ...,  0

In [15]:
# from https://pytorch-tutorials-preview.netlify.app/beginner/transformer_tutorial.html
# i don't completely understand positional encoding yet, but I have built the intuition that 
# it is analogous to how binary numbers encode numbers; smaller bits flips more frequently 
# than larger bits; this is modeled by the sinusodial waves 
# it also takes advantage of linearity of trigonometric addition formulas, which supposedly 
# helps the model to figure out relative positioning...
# https://medium.com/thedeephub/positional-encoding-explained-a-deep-dive-into-transformer-pe-65cfe8cfe10b 
class PositionalEncoding(nn.Module):

    def __init__(self, model_dim: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_dim, 2) * (-math.log(10000.0) / model_dim))
        pe = torch.zeros(max_len, 1, model_dim)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, X):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        X = X + self.pe[:X.size(0)]
        return self.dropout(X)

In [20]:
# what I need to do to finish up the encoder_decoder architecture 
# the only difference for the decoder architecture is the cross attention layer `
# which is much like the self-attension layer except that it is using both the final 
# H of the encoder and that of decoder to do query-key matching, thus decoder needs to 
# take in memory from encoder
pos = PositionalEncoding(model_dim=model_dim)


In [40]:
import dac
from ICMTSMTGuitarData import *
from jupyter_audio_utils import *

model_path = dac.utils.download(model_type="44khz")
model = dac.DAC.load(model_path, weights_only=True).eval()

mono_data2 = ICMTSMTGuitarDataMono(DEFAULT_PEDAL_PROBS)
test_2 = mono_data2[random.randint(0, len(mono_data2)-1)] 

waveform, sr = test_2[0] 
play_audio(*test_2[0])

x = model.preprocess(waveform, sr)

with torch.no_grad():
    z, codes, latents, _, _ = model.encode(x.unsqueeze(dim=0))

audio_tokens = z.transpose(-2,-1) #(batch_size, seq_len, model_dim)
N = audio_tokens.shape[-2] 
model_dim = audio_tokens.shape[-1] 
key_dim = 256
hidden_dim = 512
num_heads=4

testing = TransformerStack(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=256, num_heads=num_heads, num_stack=1)
testing(audio_tokens)

/home/yuc3/miniconda3/envs/dac_compatible/lib/python3.8/site-packages/audiotools/ml/layers/base.py:172: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = torch.load

tensor([[[ 3.9991e+00, -3.5295e+00, -2.1199e+00,  ...,  8.3164e-03,
          -1.2734e+00, -1.0567e+00],
         [ 3.0537e+00, -5.8888e+00,  1.3495e+00,  ...,  4.4167e+00,
           3.2625e+00,  2.3578e+00],
         [ 3.1654e+00, -4.8631e+00,  9.4301e-01,  ...,  3.9824e+00,
           2.7729e+00,  2.6109e+00],
         ...,
         [ 5.6230e+00, -7.2018e-01, -7.5964e+00,  ...,  2.2464e+00,
           5.5217e-01, -7.2428e-02],
         [-9.6805e-01, -3.6858e+00,  3.9257e-01,  ..., -3.7251e+00,
           1.9043e+00, -4.5633e-01],
         [ 7.8745e+00,  5.7684e-01, -1.0469e+00,  ..., -8.3304e+00,
           2.9099e+00, -1.6980e+00]]], grad_fn=<AddBackward0>)

In [ ]:
from myTransformer import *

# blueprint
class EncoderDecoder(nn.Module): 
    def __init__(
        self, 
        N, 
        model_dim,
        key_dim,
        encoder_mask,
        encoder_hidden_dim, 
        encoder_num_stack,
        encoder_num_heads,
        decoder_hidden_dim, 
        decoder_num_stack,
        decoder_num_heads,
        decoder_vocab
    ):
        pass
        self.positional_encoder = PositionalEncoding()
        self.encoder_stack = TransformerStack(
            N=N, 
            model_dim=model_dim, 
            key_dim=key_dim, 
            hidden_dim=encoder_hidden_dim, 
            num_heads=encoder_num_heads, 
            num_stack=encoder_num_stack
        )
        self.decoder_stack = TransformerStack(
            N=N, 
            model_dim=model_dim, 
            key_dim=key_dim, 
            hidden_dim=decoder_hidden_dim, 
            num_heads=decoder_num_heads, 
            num_stack=decoder_num_stack,
            cross_attention=True
        )
        self.language_head = None
        self.decoder_vocab = decoder_vocab
        self.mask = encoder_mask # do the self register thingy
    def forward(self, X): # (batch_size, N, model_dim)
        X = self.positional_encoder(X)
        H1 = self.encoder_stack(X, mask=self.mask)
        H2 = self.decoder_stack(X, H1)
        Y = self.language_head(H2,...) # not implemented yet

        return Y